In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    print('can use GPU')
    
mps_device = torch.device("mps")

can use GPU


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(mps_device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1,28,28,device=mps_device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicteed class: {y_pred}")

Predicteed class: tensor([2], device='mps:0')


In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4214, -0.1575,  0.4420,  0.0506, -0.4197,  0.3330,  0.0199,  0.1548,
         -0.1496,  0.2923,  0.2005, -0.3349,  0.2951, -0.2986,  0.4339,  0.4103,
          0.1360, -0.3234, -0.2239, -0.2042],
        [ 0.0109, -0.1044, -0.0126, -0.1174, -0.8146,  0.1880, -0.0548,  0.2158,
          0.4294,  0.6162,  0.4718, -0.1308,  0.0584, -0.1602,  0.6977,  0.1034,
          0.2960, -0.3180, -0.2281, -0.1730],
        [ 0.1976, -0.0691, -0.2336, -0.2656, -0.3828,  0.0113, -0.0946,  0.2572,
          0.3976,  0.5618,  0.0272, -0.4721,  0.2904,  0.2556,  0.2544,  0.4682,
          0.0814, -0.2635, -0.1662, -0.0676]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4214, 0.0000, 0.4420, 0.0506, 0.0000, 0.3330, 0.0199, 0.1548, 0.0000,
         0.2923, 0.2005, 0.0000, 0.2951, 0.0000, 0.4339, 0.4103, 0.1360, 0.0000,
         0.0000, 0.0000],
        [0.0109, 0.0000, 0.0000, 0.0000, 0.0000, 0.1880, 0.0000, 0.2158, 0.4294,
         0.6162, 0.4718, 0.0000, 0.0584, 0.0000, 0.69

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_tensor_str.py:106: UserWarning: The operator 'aten::masked_select' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0243,  0.0350,  0.0118,  ...,  0.0239, -0.0227, -0.0350],
        [ 0.0308,  0.0007,  0.0027,  ...,  0.0223,  0.0197,  0.0299]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0039, -0.0073], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0321,  0.0096,  0.0055,  ...,  0.0226,  0.0272,  0.0214],
        [ 0.0026,  0.0325, -0.0123,  ..., -0.0062, -0.0373, -0.0086]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([0.0093, 0.0069], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0247, -0.0060,  0.0351,  ..., -0.0159,  0.0316, -0.0165],
        [-0.0301,  0.0088, -0.0058,  ...,